In [288]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [289]:
df = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')
print(df.shape)
df.head()

In [290]:
df.isna().sum()

In [291]:
df.dtypes

In [292]:
df.columns

In [293]:
cat = []
for i in df.columns:
    if(df[i].dtype == 'object'):
        cat.append(i)
    

In [294]:
cat

In [295]:
for i in cat:
    print(i, ': ', df[i].unique())

In [296]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

labels = ['sex_label','cpt_label','restecg_label','exangina_label','st_label']

for i in range(len(cat)):
    labels[i] = OrdinalEncoder()
    #df[cat[i]] = labels[i].fit_transform(df[cat[i]])
    df[cat[i]] = labels[i].fit_transform(np.array(df[cat[i]]).reshape(-1,1))

In [297]:
df.head()

In [298]:
df.shape

In [299]:
# Annova test

from sklearn.feature_selection import f_classif

x_total = df.drop(columns=['HeartDisease'], axis=1)
y_total = df[['HeartDisease']]

fs = f_classif(x_total,y_total)


In [300]:
fs

In [301]:
fs[0][0]

In [302]:
scores = pd.DataFrame(columns=['f_score', 'p_value'])
col = []
f_score = []
p_value = []
for i in range(len(df.columns)-1):
    col.append(df.columns[i])
    f_score.append(fs[0][i])
    p_value.append(fs[1][i])

scores['column'] = col
scores['f_score'] = f_score
scores['p_value'] = p_value

scores.head()

In [303]:
scores = scores.sort_values(by=['f_score'], ascending=False)
scores = scores.reset_index(drop=True)
scores.head()

In [304]:
from matplotlib import pyplot as plt
plt.figure(figsize=(15,5))
plt.bar(scores['column'],scores['f_score'])
plt.xlabel('columns')
plt.ylabel('f_score')
plt.show()

In [305]:
plt.figure(figsize=(15,5))
plt.bar(scores['column'],scores['p_value'])
plt.xlabel('columns')
plt.ylabel('p_value')
plt.show()

In [306]:
df = df.drop(columns=['RestingBP','RestingECG'],axis=1)
df.head()

In [307]:
train = df.sample(frac=0.8, random_state=25)
test = df.drop(train.index)

print(train.shape)
print(test.shape)

In [308]:
train['HeartDisease'].value_counts()

IMbalanced dataset for classification


In [309]:
from numpy import mean
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_roc_curve

X = train.drop(columns=['HeartDisease'],axis=1)
y = train[['HeartDisease']]

def srf(X,y):
    model = RandomForestClassifier(n_estimators=10)
    
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=25)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    m = mean(scores)
    return m

def cwrf(X,y):
    model = RandomForestClassifier(n_estimators=10, class_weight='balanced')
    
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=25)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    m = mean(scores)
    return m

def cwrfs(X,y):
    model = RandomForestClassifier(n_estimators=10, class_weight='balanced_subsample')
    
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=25)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    m = mean(scores)
    return m

def brf(X,y):
    model = BalancedRandomForestClassifier()
    
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=25)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    m = mean(scores)
    return m

def eec(X,y):
    model = EasyEnsembleClassifier()
    
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=25)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    
    m = mean(scores)
    return m

In [310]:
#srf = srf(X,y)
#print('SRF:',' ', srf)

In [311]:
#cwrf = cwrf(X,y)
#print('CWRF:',' ',cwrf)

In [312]:
#cwrfs = cwrfs(X,y)
#print('CWRF:',' ',cwrfs)

In [313]:
#brf = brf(X,y)
#print('BRf:',' ',brf)

In [314]:
#eec = eec(X,y)
#print('EEC:',' ',eec)

In [315]:

model = BalancedRandomForestClassifier()
    
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=25)
#scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    
m = mean(scores)

model.fit(X,y)

pred = model.predict(test.drop(columns=['HeartDisease'],axis=1))
#pred

acc = accuracy_score(test[['HeartDisease']], pred)
print('accuracy: ', acc)
plot_roc_curve(model, test.drop(columns=['HeartDisease'],axis=1), test[['HeartDisease']])

In [316]:
from sklearn.model_selection import GridSearchCV

model1 = BalancedRandomForestClassifier()

param_grid = [{'n_estimators':[3,10,30]}]

grid_search = GridSearchCV(model1, param_grid, scoring = 'roc_auc', cv=5, return_train_score=True)

grid_search.fit(X,y)



In [317]:
grid_search.best_params_

In [318]:
grid_search.best_estimator_

In [319]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres['mean_test_score'],cvres['params']):
    print(mean_score,params)

In [320]:
p3 = grid_search.predict(test.drop(columns=['HeartDisease'],axis=1))
ac1 = accuracy_score(test[['HeartDisease']], p3)
print('accuracy: ', ac1)

In [321]:

model2 = BalancedRandomForestClassifier(n_estimators=30)
    
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=25)
#scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)

model2.fit(X,y)

pred1 = model2.predict(test.drop(columns=['HeartDisease'],axis=1))
#pred

acc1 = accuracy_score(test[['HeartDisease']], pred1)
print('accuracy: ', acc1)
plot_roc_curve(model2, test.drop(columns=['HeartDisease'],axis=1), test[['HeartDisease']])

In [323]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(test[['HeartDisease']], pred1, labels=model2.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model2.classes_)
disp.plot()
plt.show()